In [1]:
import numpy as np
import pandas as pd

### 下图是本次代码生成之后架构，大家可以根据该架构自行裁剪使用

![PAI](https://user-images.githubusercontent.com/21167490/37100909-3536a2b8-225f-11e8-98de-903da56e7a9d.png)

### 本脚本主要是是快速生成PAI命令行，模型参数大家可以根据官网介绍自行调节尝试

In [ ]:
=> ycc_sales_lichuan0307_trainset_1_time_window 训练集 201301-201709        
--      => ycc_sales_lichuan0307_testset_1_time_window  测试集 201710  
--      => ycc_sales_lichuan0307_trainset_2_time_window 训练集 201302-201710        
--      => ycc_sales_lichuan0307_testset_2_time_window  测试集 201711    
--      => ycc_sales_lichuan0307_trainset_3_time_window 训练集 201303-201711        
--      => ycc_sales_lichuan0307_testset_3_time_window  测试集 201712   

LiChuan0131FeatureEngineer_study.ipynb
LiChuan0201SaleDateEngineer_study.ipynb
LiChuan0222SplitTrainSet.ipynb
PAI命令脚本架构.rp
README.md
S2_PAI_woody_0307_xgboost_gbdt_script_v1-Copy1.0.ipynb
S2_PAI_woody_0307_xgboost_gbdt_script_v1.0.ipynb
S2_chaos.ipynb
Wood_0209_part_optimize__LiChuan0207XGBoost_.ipynb
Wood_20180224_lstm_predict_only_classid.ipynb
Woody0204car_sale.ipynb
Woody0205car_sale_mean_3Months.ipynb
Woody_20180204_lstm_predict_December.ipynb
Woody_20180224_keras_practice.ipynb
Woody_20180224_lstm_predict-all_features.ipynb
Woody_20180224_lstm_predict_add_displacement.ipynb
Woody_20180224_lstm_predict_add_price_feature.ipynb
Woody_20180225_svm_practice_R.ipynb
Woody_20180226_Observe-Nov-vs-Dec-quantity.ipynb
Woody_20180304_Observe-Dec-vs-Jan-quantity.ipynb
XGboost_param_share.ipynb
[new] yancheng_train_20171226.csv
caijun0208TimefeatureEngineering-Copy1.ipynb
caijun0222ResultStack.ipynb
pandas_groupby.ipynb
woody-20180226-Observe-classid-quantity-compare.ipynb
yancheng_testA_2017

In [88]:


#imput_table_train:需要训练的表名称，数量依照情况而定
imput_table_train=['ycc_sales_lichuan0307_trainset_1_time_window','woody_imput_train_1','ycc_sales_lichuan0307_trainset_2_time_window','ycc_sales_lichuan0307_trainset_3_time_window'] #4个

#imput_table_test：测试表的名称
imput_table_test=['ycc_sales_lichuan0307_testset_1_time_window','ycc_sales_lichuan0307_testset_2_time_window','ycc_sales_lichuan0307_testset_3_time_window']

#imput_table_predict：预测表的名称
imput_table_predict=['woody_imput_predict_0']

#output_model：模型训练完之后保存模型的名称，之后用于预测 测试表或者最终测试表
output_model=['woody_output_model_0','woody_output_model_1','woody_output_model_2','woody_output_model_3'] #4个

#test_output_table:是测试集输出的结果，为之后评估模型做准备数据
test_output_table=['woody_test_out_table_0','woody_test_out_table_1','woody_test_out_table_2']

#estimate_output_table：主要是用来评估结果结果的输出表
estimate_output_table=['woody_estimate_output_table_0','woody_estimate_output_table_1','woody_estimate_output_table_2']#3个


#predict_output_table：主要是用来预测最终结果集生成的表
predict_output_table=['woody_predict_output_table_0','woody_predict_output_table_1','woody_predict_output_table_2'] #3个


# train_col：是训练模型的输入列 也是 预测时候的输入列

train_col_0='province_id,city_id,class_id'
train_col_1='province_id,city_id,class_id'
train_col_2='province_id,city_id,class_id'
train_col_3='province_id,city_id,class_id'
train_col=[train_col_0,train_col_1,train_col_2,train_col_3]
# 模型融合系数
ensemble_number=[0.2,0.3,0.5]

# xgboost模型

In [73]:
#训练模型
xgboost_train='''
PAI --xgboost训练模型第%s个
-name xgboost -project algo_public
-Deta="0.01"
-Dobjective="reg:linear"
-DitemDelimiter=","
-Dseed="0"
-Dnum_round="1000"
-DlabelColName="sale_quantity"
-DinputTableName="%s"--输入表
-DenableSparse="false"
-Dmax_depth="8"
-Dsubsample="1"
-Dcolsample_bytree="1"
-DmodelName="%s" --模型名称
-Dgamma="0"
-Dlambda="50"
-DfeatureColNames="%s" --训练feature
-Dbase_score="0.5"
-Dmin_child_weight="100"
-DkvDelimiter=":";
'''

In [74]:
#测试集
xgboost_test='''
PAI --测试集第%s个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="%s"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="%s"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="%s" --输入的表名
-DenableSparse="false" --输入表是否为稀疏格式
-DappendColNames="sale_date,province_id,city_id,class_id,sale_quantity" --输出表中保留输入表的列名
--输入表中哪些列作为预测的feature，多列以逗号分隔,默认所有列,该值与训练脚本的DfeatureColNames值一样
-DfeatureColNames="%s";
'''

In [75]:
#预测集
xgboost_predict='''
PAI --最终结果预测集第%s个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="%s"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="%s"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="%s" --输入的表名
-DenableSparse="false" --输入表是否为稀疏格式
-DappendColNames="sale_date,province_id,city_id,class_id,sale_quantity" --输出表中保留输入表的列名
--输入表中哪些列作为预测的feature，多列以逗号分隔,默认所有列,该值与训练脚本的DfeatureColNames值一样
-DfeatureColNames="%s";
'''

In [76]:
#评估集，只对测试集评估
xgboost_estimate='''
PAI --回归评估第%s个
-name regression_evaluation -project algo_public
-DinputTableName="%s"--输入表名
-DyColName="sale_quantity"   --输入表lable列
-DpredictionColName="prediction_result"  --预测结果列
-DindexOutputTableName="%s"  --输出表表名
-Dlifecycle=28
-DresidualOutputTableName="%s_histogram";-- 残差直方图输出表表名，必填
'''

In [90]:
#模型融合
merge_table='''
--合并表一
drop table if exists stuff_woody_0307_test;
create table if not exists stuff_woody_0307_test
as SELECT a.sale_date, a.province_id, a.city_id, a.class_id,a.prediction_result*%s+b.prediction_result*%s as prediction_result
FROM (SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) a
JOIN
(SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) b
ON( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);

--合并表二
drop table if exists stuff_woody_0307_test_new;
create table if not exists stuff_woody_0307_test_new
as SELECT a.sale_date, a.province_id, a.city_id, a.class_id,a.prediction_result+b.prediction_result*%s as prediction_result
FROM (SELECT sale_date, province_id, city_id, class_id, prediction_result FROM stuff_woody_0307_test) a
JOIN
(SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) b
ON( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);


--创建新表匹配最终表的数据
drop table if exists yc_result_submit_a_copy;
create table if not exists yc_result_submit_a_copy as
SELECT a.predict_date,a.province_id, a.city_id, a.class_id,pow(2,b.prediction_result)-1 as predict_quantity
FROM yc_result_submit_a a LEFT OUTER JOIN stuff_woody_0307_test_new b
ON ( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);

-- 需要把表yc_result_submit_a和yc_result_submit_a_copy换一下
drop table if exists yc_result_submit_a;
create table if not exists yc_result_submit_a as select * from yc_result_submit_a_copy;
'''

In [91]:
#查看生成结果
for i in range(len(imput_table)):
    print(xgboost_train%(i,imput_table_train[i],output_model[i],train_col[i]))
    if i < len(imput_table)-1:
        print(xgboost_test%(i,output_model[i],test_output_table[i],imput_table_test[i],train_col[i]))
        print(xgboost_estimate%(i,test_output_table[i],estimate_output_table[i],estimate_output_table[i]))
    if i !=0:
        print(xgboost_predict%(i-1,output_model[i],predict_output_table[i-1],imput_table_predict[0],train_col[i]))
print(merge_table%(ensemble_number[0],ensemble_number[1],predict_output_table[0],predict_output_table[1],ensemble_number[2],predict_output_table[2]))    



PAI --xgboost训练模型第0个
-name xgboost -project algo_public
-Deta="0.01"
-Dobjective="reg:linear"
-DitemDelimiter=","
-Dseed="0"
-Dnum_round="1000"
-DlabelColName="sale_quantity"
-DinputTableName="woody_imput_train_0"--输入表
-DenableSparse="false"
-Dmax_depth="8"
-Dsubsample="1"
-Dcolsample_bytree="1"
-DmodelName="woody_output_model_0" --模型名称
-Dgamma="0"
-Dlambda="50"
-DfeatureColNames="province_id,city_id,class_id" --训练feature
-Dbase_score="0.5"
-Dmin_child_weight="100"
-DkvDelimiter=":";


PAI --测试集第0个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="woody_output_model_0"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="woody_test_out_table_0"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="woody_imput_test_0" --输入的表名
-DenableSparse="f

In [92]:
#保存文本
path='../../src/SQL/woody_0307_xgboost_train_and_predict.sql'
with open(path,'w') as file:
    for i in range(len(imput_table)):
        file.write(xgboost_train%(i,imput_table_train[i],output_model[i],train_col[i]))
        if i < len(imput_table)-1:
            file.write(xgboost_test%(i,output_model[i],test_output_table[i],imput_table_test[i],train_col[i]))
            file.write(xgboost_estimate%(i,test_output_table[i],estimate_output_table[i],estimate_output_table[i]))
        if i !=0:
            file.write(xgboost_predict%(i-1,output_model[i],predict_output_table[i-1],imput_table_predict[0],train_col[i]))
    file.write(merge_table%(ensemble_number[0],ensemble_number[1],predict_output_table[0],predict_output_table[1],ensemble_number[2],predict_output_table[2]))    



In [93]:
#再次确认
with open(path) as file:
    print(file.read())


PAI --xgboost训练模型第0个
-name xgboost -project algo_public
-Deta="0.01"
-Dobjective="reg:linear"
-DitemDelimiter=","
-Dseed="0"
-Dnum_round="1000"
-DlabelColName="sale_quantity"
-DinputTableName="woody_imput_train_0"--输入表
-DenableSparse="false"
-Dmax_depth="8"
-Dsubsample="1"
-Dcolsample_bytree="1"
-DmodelName="woody_output_model_0" --模型名称
-Dgamma="0"
-Dlambda="50"
-DfeatureColNames="province_id,city_id,class_id" --训练feature
-Dbase_score="0.5"
-Dmin_child_weight="100"
-DkvDelimiter=":";

PAI --测试集第0个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="woody_output_model_0"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="woody_test_out_table_0"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="woody_imput_test_0" --输入的表名
-DenableSparse="fa

# GBDT模型

In [94]:
#训练模型
gbdt_train='''
PAI --gbdt训练模型第%s个
-name GBDT -project algo_public
-DfeatureSplitValueMaxSize="500"
-DlossType="3" --3-REGRESSION LOSS：mean squared error loss
-DrandSeed="0"
-DnewtonStep="1"
-Dshrinkage="0.5"
-DmaxLeafCount="32"
-DlabelColName="sale_quantity"
-DinputTableName="%s" --输入表
-DminLeafSampleCount="1"
-DsampleRatio="1"
-DmaxDepth="10"
-DmetricType="0"
-DmodelName="%s" --模型名称
-DfeatureRatio="1"
-Dp="1" --可选）指数底数。gbrank与regression loss中的指数底数,默认1如果p 1,将样本的label映身为p^label；当p <= 1时，不做映射，保持原来的label.范围1到5
-Dtau="0.6" --Tau参数。gbrank loss中的Tau参数，默认0.6，范围：[0,1]
-DtestRatio="0"
-DfeatureColNames="%s"--训练feature
-DtreeCount="50";
'''

In [95]:
#测试集
gbdt_test='''
PAI --测试集第%s个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="%s"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="%s"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="%s" --输入的表名
-DenableSparse="false" --输入表是否为稀疏格式
-DappendColNames="sale_date,province_id,city_id,class_id,sale_quantity" --输出表中保留输入表的列名
--输入表中哪些列作为预测的feature，多列以逗号分隔,默认所有列,该值与训练脚本的DfeatureColNames值一样
-DfeatureColNames="%s";
'''

In [96]:
#预测集
gbdt_predict='''
PAI --最终结果预测集第%s个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="%s"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName="%s"--输出表的名
-DscoreColName="prediction_score" --输出表中score列名
-DkvDelimiter=":" --当输入表数据为稀疏格式时，key和value的分割符
-DinputTableName="%s" --输入的表名
-DenableSparse="false" --输入表是否为稀疏格式
-DappendColNames="sale_date,province_id,city_id,class_id,sale_quantity" --输出表中保留输入表的列名
--输入表中哪些列作为预测的feature，多列以逗号分隔,默认所有列,该值与训练脚本的DfeatureColNames值一样
-DfeatureColNames="%s";
'''

In [97]:
#评估集，只对测试集评估
gbdt_estimate='''
PAI --回归评估第%s个
-name regression_evaluation -project algo_public
-DinputTableName="%s"--输入表名
-DyColName="sale_quantity"   --输入表lable列
-DpredictionColName="prediction_result"  --预测结果列
-DindexOutputTableName="%s"  --输出表表名
-Dlifecycle=28
-DresidualOutputTableName="%s_histogram";-- 残差直方图输出表表名，必填
'''

In [98]:
#模型融合
merge_table='''
--合并表一
drop table if exists stuff_woody_0307_test;
create table if not exists stuff_woody_0307_test
as SELECT a.sale_date, a.province_id, a.city_id, a.class_id,a.prediction_result*%s+b.prediction_result*%s as prediction_result
FROM (SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) a
JOIN
(SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) b
ON( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);

--合并表二
drop table if exists stuff_woody_0307_test_new;
create table if not exists stuff_woody_0307_test_new
as SELECT a.sale_date, a.province_id, a.city_id, a.class_id,a.prediction_result+b.prediction_result*%s as prediction_result
FROM (SELECT sale_date, province_id, city_id, class_id, prediction_result FROM stuff_woody_0307_test) a
JOIN
(SELECT sale_date, province_id, city_id, class_id, prediction_result FROM %s) b
ON( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);


--创建新表匹配最终表的数据
drop table if exists yc_result_submit_a_copy;
create table if not exists yc_result_submit_a_copy as
SELECT a.predict_date,a.province_id, a.city_id, a.class_id,pow(2,b.prediction_result)-1 as predict_quantity
FROM yc_result_submit_a a LEFT OUTER JOIN stuff_woody_0307_test_new b
ON ( a.province_id=b.province_id and a.city_id=b.city_id and a.class_id=b.class_id);

-- 需要把表yc_result_submit_a和yc_result_submit_a_copy换一下
drop table if exists yc_result_submit_a;
create table if not exists yc_result_submit_a as select * from yc_result_submit_a_copy;
'''

In [99]:
#查看生成结果
for i in range(len(imput_table)):
    print(gbdt_train%(i,imput_table_train[i],output_model[i],train_col[i]))
    if i < len(imput_table)-1:
        print(gbdt_test%(i,output_model[i],test_output_table[i],imput_table_test[i],train_col[i]))
        print(gbdt_estimate%(i,test_output_table[i],estimate_output_table[i],estimate_output_table[i]))
    if i !=0:
        print(gbdt_predict%(i-1,output_model[i],predict_output_table[i-1],imput_table_predict[0],train_col[i]))
print(merge_table%(ensemble_number[0],ensemble_number[1],predict_output_table[0],predict_output_table[1],ensemble_number[2],predict_output_table[2]))    


PAI --gbdt训练模型第0个
-name GBDT -project algo_public
-DfeatureSplitValueMaxSize="500"
-DlossType="3" --3-REGRESSION LOSS：mean squared error loss
-DrandSeed="0"
-DnewtonStep="1"
-Dshrinkage="0.5"
-DmaxLeafCount="32"
-DlabelColName="sale_quantity"
-DinputTableName="woody_imput_train_0" --输入表
-DminLeafSampleCount="1"
-DsampleRatio="1"
-DmaxDepth="10"
-DmetricType="0"
-DmodelName="woody_output_model_0" --模型名称
-DfeatureRatio="1"
-Dp="1" --可选）指数底数。gbrank与regression loss中的指数底数,默认1如果p 1,将样本的label映身为p^label；当p <= 1时，不做映射，保持原来的label.范围1到5
-Dtau="0.6" --Tau参数。gbrank loss中的Tau参数，默认0.6，范围：[0,1]
-DtestRatio="0"
-DfeatureColNames="province_id,city_id,class_id"--训练feature
-DtreeCount="50";


PAI --测试集第0个
-name prediction -project algo_public
-DdetailColName="prediction_detail"  --输出表中detail列名
-DmodelName="woody_output_model_0"--输入模型名称
-DitemDelimiter="," --当输入表数据为稀疏格式时，kv间的分割符，这个和DkvDelimiter有什么区别？
-DresultColName="prediction_result" --输出表result列名称
-Dlifecycle="28" --指定输出表的生命周期，应该是28天吧
-DoutputTableName

In [ ]:
#保存文本
path='../../src/SQL/woody_0307_gbdt_train_and_predict.sql'
with open(path,'w') as file:
    for i in range(len(imput_table)):
        file.write(gbdt_train%(i,imput_table_train[i],output_model[i],train_col[i]))
        if i < len(imput_table)-1:
            file.write(gbdt_test%(i,output_model[i],test_output_table[i],imput_table_test[i],train_col[i]))
            file.write(gbdt_estimate%(i,test_output_table[i],estimate_output_table[i],estimate_output_table[i]))
        if i !=0:
            file.write(gbdt_predict%(i-1,output_model[i],predict_output_table[i-1],imput_table_predict[0],train_col[i]))
    file.write(merge_table%(ensemble_number[0],ensemble_number[1],predict_output_table[0],predict_output_table[1],ensemble_number[2],predict_output_table[2]))    

In [ ]:
#再次确认
with open(path) as file:
    print(file.read())